<a href="https://colab.research.google.com/github/kaziunt2022/Assignment_03/blob/main/INFO5731_Assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INFO5731 Assignment Three**

In this assignment, you are required to conduct information extraction, semantic analysis based on **the dataset you collected from assignment two**. You may use scipy and numpy package in this assignment. (Use 500 records amount all the data you collected).

# **Question 1: Understand N-gram**

(45 points). Write a python program to conduct N-gram analysis based on the dataset in your assignment two:

(1) Count the frequency of all the N-grams (N=3).

(2) Calculate the probabilities for all the bigrams in the dataset by using the fomular count(w2 w1) / count(w2). For example, count(really like) / count(really) = 1 / 3 = 0.33.

(3) Extract all the **noun phrases** and calculate the relative probabilities of each review in terms of other reviews (abstracts, or tweets) by using the fomular frequency (noun phrase) / max frequency (noun phrase) on the whole dataset. Print out the result in a table with column name the all the noun phrases and row name as the 500 reviews (abstracts, or tweets). 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
# Write your code here
# Import Libraries
import pandas as pd
import nltk, string, re
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
## Importing the data
df = pd.read_csv("https://raw.githubusercontent.com/kaziunt2022/Assignment_03/main/Reviews.CSV", nrows=500) # import first 500 reviews
all_reviews = list(df['Reviews']) # extracting reviews text

## Tokenize the text of reviews
review_tokens = [nltk.tokenize.casual_tokenize(i) for i in all_reviews]

## Cleaning the data
stopwords = nltk.corpus.stopwords.words('english') # get all stopwords
punctuations = string.punctuation # get all punctuations
clean_text = [j.lower() for i in review_tokens for j in i if ((j not in stopwords) and (j not in punctuations) and (j.isalpha()))] # removing all punctions, stopwords, and non alphabatic words

## get 3 grams
three_grams = nltk.ngrams(clean_text, 3)
freq_three_word = {}
# Count freq of each tri-gram
for three_word in three_grams:
    if three_word in freq_three_word:
        freq_three_word[three_word]+=1
    else:
        freq_three_word[three_word]=1
print("Part 1:\n", freq_three_word)
print()

## calculate probability
# claculate frequency of each word
freq_each_word = {}
for word in clean_text:
    if word in freq_each_word:
        freq_each_word[word]+=1
    else:
        freq_each_word[word]=1
# find 2 gram and calculate there frequency
two_gram = nltk.ngrams(clean_text, 2)
freq_two_word = {}
for two_word in two_gram:
    if two_word in freq_two_word:
        freq_two_word[two_word]+=1
    else:
        freq_two_word[two_word]=1
# find out probability of each bigram
probability_two_gram = {}
for two_gram_words in freq_two_word:
    probability_two_gram[two_gram_words] = freq_two_word[two_gram_words]/freq_each_word[two_gram_words[0]]
# print out probability of each bigram
print("Part 2\n", probability_two_gram)
print()

## NOUN PRBABILITY
# get all the noun phrases
noun_phrases, count = "", 0
for word, tag in nltk.pos_tag(clean_text): # get tags of each words
    if re.match(r"NN.*", tag): # get all the nouns
        count+=1
        if count>=1: noun_phrases = noun_phrases + word + " " # add to the noun_phrase is the noun us present in the liist
    else: noun_phrases, count = noun_phrases+"-----", 0
noun_phrases = re.sub(r"-+","?",noun_phrases).split("?") # seperating the npoun pharses in
noun_phrases = [i.strip() for i in noun_phrases if i!=""]

# calculte the frequency of the noun phrases
noun_phrase_freq = {}
for nPhrase in noun_phrases:
    if nPhrase in noun_phrase_freq: noun_phrase_freq[nPhrase]+=1
    else: noun_phrase_freq[nPhrase]=1
# Calculate probality of the noun phrases
max_freq = max(noun_phrase_freq.values()) # max noun phrase freq
noun_phrase_probability = {}

for nPhraseFreq in noun_phrase_freq:
    noun_phrase_probability[nPhraseFreq] = noun_phrase_freq[nPhraseFreq]/max_freq # noun phrase freq

nounPhraseProbability_df = pd.DataFrame(noun_phrase_probability.items(), columns=['Noun Phrase', 'Probability']) # making up dataframe of the probability
print("Part 3\n", nounPhraseProbability_df) # print out noun phrase probabilty
print()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Part 1:
 {('start', 'saying', 'looking'): 1, ('saying', 'looking', 'great'): 1, ('looking', 'great', 'story'): 1, ('great', 'story', 'disappointed'): 1, ('story', 'disappointed', 'pretty'): 1, ('disappointed', 'pretty', 'standard'): 1, ('pretty', 'standard', 'journey'): 1, ('standard', 'journey', 'core'): 1, ('journey', 'core', 'shame'): 1, ('core', 'shame', 'story'): 1, ('shame', 'story', 'could'): 1, ('story', 'could', 'inspired'): 1, ('could', 'inspired', 'house'): 1, ('inspired', 'house', 'flying'): 1, ('house', 'flying', 'daggers'): 2, ('flying', 'daggers', 'wuxia'): 1, ('daggers', 'wuxia', 'rating'): 1, ('wuxia', 'rating', 'high'): 1, ('rating', 'high', 'because'): 1, ('high', 'because', 'story'): 1, ('because', 

# **Question 2: Undersand TF-IDF and Document representation**

(40 points). Starting from the documents (all the reviews, or abstracts, or tweets) collected for assignment two, write a python program: 

(1) To build the **documents-terms weights (tf*idf) matrix bold text**.

(2) To rank the documents with respect to query (design a query by yourself, for example, "An Outstanding movie with a haunting performance and best character development") by using **cosine similarity**.

In [2]:
# Write your code here
#Import Libraries
import nltk, string, re
import pandas as pd
from collections import Counter
import numpy as np

nltk.download('punkt')

query = "An Outstanding movie with a haunting performance and best character development" # Query Sentence

reviews = list(pd.read_csv("https://raw.githubusercontent.com/kaziunt2022/Assignment_03/main/Reviews.CSV", nrows=500)["Reviews"]) # get reviews
total_seneteces = len(reviews) # total number of reviews

def clean_txt(txt):
    """
    Input: txt --> str
    Func: Clean the given text
    Output: list of clean text
    """
    words = nltk.tokenize.word_tokenize(txt) # tokenize txt
    stopwords = nltk.corpus.stopwords.words('english') # get all stopwords
    punctuations = string.punctuation # get all punctuations
    clean_text = [j.lower() for j in words if ((j not in stopwords) and (j not in punctuations) and (j.isalpha()))] # removing all punctions, stopwords, and non alphabatic words
    return clean_text

def get_td_idf(reviews):
    """
    Input: reviews --> list: list of all reviews
    Func: find tf, idf, word weight, words weight dataframe, word weight array
    Output: list of tf, idf, word weight, words weight dataframe, word weight array
    """
    tf = []
    idf = {}
    for review in reviews:
        text_counter = Counter(review) # count freq of each word
        counted_words = []
        sentence_tf = {}
        total_words = len(review)
        for word in review:
            if word not in counted_words: # go through each word of review
                sentence_tf[word] = text_counter[word]/total_words # store tf value of each word
                counted_words.append(word)
            if word not in idf:  
                count = 1
                for sentence in reviews: # check the presence of each word in whole document
                    if word in sentence: 
                        count+=1
                idf[word]= np.log10(total_seneteces/count) # store idf value of each word 
        tf.append(sentence_tf)

    word_weight = []
    for sentence_tf in tf:
        sentence_weight = {}
        for word in sentence_tf:
            sentence_weight[word] = sentence_tf[word] * idf[word] # Calculate sentence weight
        word_weight.append(sentence_weight)

    words_weight_dataframe = pd.DataFrame(word_weight) # make dataframe of sentence weight
    word_weight_array = words_weight_dataframe.values

    return [tf, idf, word_weight, words_weight_dataframe, word_weight_array] 

clean_reviews = [clean_txt(i) for i in reviews] # clean all reviews

[tf, idf, word_weight, words_weight_dataframe, word_weight_array] = get_td_idf(clean_reviews)  # apply get_td_idf on all reviews

q_clean = clean_txt(query)  # clean query sentence
[q_tf, q_idf, q_word_weight, q_words_weight_dataframe, q_word_weight_array] = get_td_idf([q_clean]) # apply get_td_idf on query

def cosine_similarity(tfidf_query, tfidf_reviews , query , doc_num):
    """
    Func: find cosin similarity between reviews as given query
    """
    dot_product = 0
    q_mod = 0
    doc_mod = 0
   
    for keyword in query: # go through each word of query
        try: query_score = tfidf_query[0][keyword] # try to find query word if not it suppose to be zero
        except: query_score = 0
        try: doc_score = tfidf_reviews[doc_num][keyword] # try to find reviews word if not it suppose to be zero
        except: doc_score=0
        
        dot_product += query_score * doc_score # dot product of query and review score
        q_mod += query_score**2 # magnitude of query score
        doc_mod += doc_score**2 # magnitude of review score
    q_mod = np.sqrt(q_mod) 
    doc_mod = np.sqrt(doc_mod)
    cosine = float(dot_product/(q_mod * doc_mod)) # find cosin value
    if str(cosine)=='nan':
        return 0
    return cosine
    

cosin_sim = {}

for n in range(len(reviews)): # go through each review and find cosin simalirty between review and query and store it in a dict
    cosin_sim[reviews[n]] = cosine_similarity(q_word_weight, word_weight, q_clean, n)

review_ranking = dict(sorted(cosin_sim.items(),key=lambda x:x[1],reverse = True)) # sort the dict

review_ranking_df = pd.DataFrame(review_ranking.items(), columns=['Review', 'Cosin Similarity']) # store the data in a dataframe

review_ranking_df.to_csv('Review Ranking.CSV', index = False) # save the data frame



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:90: RuntimeWarning: invalid value encountered in double_scalars


# **Question 3: Create your own training and evaluation data for sentiment analysis**

(15 points). **You dodn't need to write program for this question!** Read each of the 500 review (if you collected the data from citeseer, you can think of other text classfication tasks instead of sentiment analysis) you collected in detail, and annotate each review with a sentiment (positive, negative, or neutral). Save the annotated dataset into a csv file with three columns (first column: document_id, clean_text, sentiment), upload the csv file to GitHub and submit the file link blew. This datset will be used for assignment four: sentiment analysis and text classification. 


In [3]:
# The GitHub link of your final csv file
url = "https://github.com/kaziunt2022/Assignment_03/blob/main/review_sentiments.CSV"
# Link: 